<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_KMeanVsBERT_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SUBJECT: K-Mean Vs BERT Embedding

###AUTHOR: Sheida Talei

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [9]:
import transformers
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report





In [10]:


from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import TFBertModel, TFBertPreTrainedModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features, InputExample

In [1]:
# !pip install --target=$nb_path flair

  Using cached https://files.pythonhosted.org/packages/68/a0/a1b41fa2fcb23ff71ba9148af75211dcccc35b256dea821b36e1ee871848/flair-0.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/63/98858cbead27df7536c7e300c169da0999e9704d02220dc6700b804eeff0/Janome-0.4.1-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b/ftfy-5.8-cp36-none-any.whl
Processing /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b/segtok-1.5.10-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3a/83/e74092e7f24a08d751aa59b37a9fc572b2e4af3918cb66f7766c3affb1b4/transformers-3.5.1-py3-none-any.whl
Processing /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57/langdetect-1.0.8-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/a4/d0a884c4300004a78cca907a6ff9a5e9fe4f090f5d95ab341c53d28cbc58/sentencepiece-0.1.91-cp36-cp36m-many

In [2]:
from flair.embeddings import BertEmbeddings

In [ ]:
from nltk import tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [12]:
X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')
Y_train= X_and_Y.Label
X_train = X_and_Y.text

In [ ]:

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
# MAX_LEN = 150

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
MAX_LEN = 64

In [ ]:
def preprocessing_for_bert(data):
  input_ids = []
  attention_masks = []
  for row in data:
    sents = tokenize.sent_tokenize(row)
    print(sents)
    for sent in sents:
      encoded_sent = tokenizer.encode_plus(text=sent,
                                          add_special_tokens=True,
                                          max_length=MAX_LEN,
                                          pad_to_max_length=True,
                                          return_attention_mask=True,
                                          truncation=True)
      input_ids.append(encoded_sent.get('input_ids'))
      attention_masks.append(encoded_sent.get('attention_mask'))
  # Convert lists to tensors
  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)

  return input_ids, attention_masks


tokens_tensor, segments_tensor = preprocessing_for_bert(X_train)

['یار طلبید#طلب مونس زیاد جان دولت حاجت ماست صحبت بس ان']
['اخور توبره میخوره حکومت']
['شکاف نیز تشدید شورا خاتم گفتمان بین ولایت جمهوری غلبه احمد ذیل انتخاباتهاست تحلیل نصب عضویت عمیق صدر استصواب جامعه اسلام ایت نگهبان سید نشانه فقیه نظارت الله انتخاب نظریه']
['مار روسرم باغ گذاشت#گذار خدا بچه پا']
['ابادان نفر میسپاریم دسترس داشت#دار سه']
['عزادار پورن ناموس جهانی لعنت کرد#کن استار هشتگ وطن راحت خارج گذر قماش']
['زدن پلیس ایجاد نوامیس ویدیو گزارش صدا امن سپهر مکرر شهروند خودرو تخریب کردن تجاوز خانواده متجاهری معتاد اتش تلفن تهدید محله مهرداد دندان صد مستند خورد#خور نا صفر']
['زن امنیت خوش رژیم نخواهند دید']
['وحشت رسیده است غنی اشرف قتل انتها']
['کشته گریه کاررو روبرو ک کشت#کش عصبانی خودمو عمه خودشو التماس سال حلق شوهر اب از داشت#دار اویز زیاد خوابشو اشاره کردی دوسش بود#باش پرسید#پرس میکرد خیلی']
['امریکا میشیگان دیترویت ایالت بسیارست خیابان امدند ارا شعار کرد#کن افراد ثبت مرده دزدی متوقف']
['مادر اکرم اوریم 💀🇮🇱 مقدس ص صهیونیست سوزانیم حساب پدر دزد چشم دفاع پیامبر 👎 ساحت بچه داشته با

/content/notebooks/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Streaming output truncated to the last 5000 lines.
['امریکا پیروز انتخابات می شود']
['پیدا عکسایی فاصله اهای کرد#کن گذاشت#گذار مشخص اینارم گشت#گرد اونایی دید#بین نشه تو کاش… مراسم']
['ازادی تلویزیون ابان زنده سیما ایران پخش ملی']
['تعداد موجود اعلام کاهش محصولات زیاد داشت#دار فروش یکباره چیه گذاشت#گذار دیروز قیمت']
['بیعت درمان ای عید برپایی عده علاج تشکر درد ناپذیرست']
['امریکا تلاش ملی رژیم امنیت مشاور دخالت کاخ انتخابات سفید']
['میشوندوبعضی بازی بعضی بخصوص می کنند مطرح ورفقایش دام حقه مسئله ازیکسو ازسوی نیفتادن تله احمدی نژادی هایند باهدف ودوستانش اقایان احمدی نژادنیفتند عبد اینکه یااستعفای اسیر استیضاح رسا هرکدام']
['امریکا اخر دخالت های رقیب دموکراتش مناظره چاره جمهوری شاید مونارنجی ریاست جمهوریخواهان داشت#دار زیاد انتخابات دیگر مهار مشکلات کمیسیون مکرر اخلاق برگزاری میکروفون خاموش ایجاد حرف های نامزد']
['ادرس جعلی جهاد میشود دست افغان قربان تصویر اینسو فضا روز دولت ایران همدرد قدر ملاهای مجاز فتوا گرم همسایه استقبال دانست#دان عوض جاهل بدست صادرنمودن قدرشناس خیلی']
['امن شهر اخوند

In [ ]:
print(tokens_tensor.shape)

NameError: ignored

In [14]:
#Loading the model

import os
checkpoint_path = "/content/drive/MyDrive/Final/Bert1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels = 2, output_hidden_states = True)
# model = model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')



Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.compile(optimizer=optimizer, loss=loss, metrics=[ metric_accuracy])
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)
# model.load_weights(checkpoint_path)


In [21]:
#test
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)

In [17]:
# model.eval()

In [ ]:
# can use last hidden state as word embeddings
#source:https://medium.com/analytics-vidhya/bert-word-embeddings-deep-dive-32f6214f02bf
# model.eval()
# import torch
# with torch.no_grad():
#   outputs = model(tokens_tensor, segments_tensor)



In [ ]:

# last_hidden_state = output[0]

In [28]:

# bert_embedding = BertEmbeddings(bert_model_or_path=model)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-multilingual-cased')

Exception when trying to download https://sbert.net/models/bert-base-multilingual-cased.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/bert-base-multilingual-cased.zip not found. Try to create it from scratch
Try to create Transformer Model bert-base-multilingual-cased with mean pooling


In [29]:
#Source: https://github.com/UKPLab/sentence-transformers
sentence_embeddings = model.encode(X_train)

In [34]:
# sentence_embeddings[0]

In [35]:
# !pip install --target=$nb_path kneed

In [33]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
from kneed import KneeLocator

In [ ]:
SSE = []
for cluster in range(1,50):
    kmeans = KMeans(n_jobs = -1, n_clusters = cluster, init='k-means++')
    kmeans.fit(sentence_embeddings)
    SSE.append(kmeans.inertia_)
    print (cluster)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,50), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

1
2
3
4


In [ ]:
# frame = pd.DataFrame({'Cluster':range(1,50), 'SSE':SSE})
# plt.figure(figsize=(12,6))
# plt.plot(frame['Cluster'], frame['SSE'], marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')

In [ ]:
kl = KneeLocator(range(1, 50), SSE, curve="convex", direction="decreasing")
true_k = kl.elbow
print (true_k)

In [ ]:
kmean=KMeans(n_clusters=true_k)
kmean.fit(X_train_vector)

In [ ]:
#Source: https://pythonprogramminglanguage.com/kmeans-text-clustering/
print("Top terms per cluster:")
order_centroids = kmean.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :50]:
        print(' %s' % terms[ind]),
    print